# Perfil de comércio

## Setup

### Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

### Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = 
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


### Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


### Inicializar a API e transferir os códigos usados

In [3]:
import os
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)
comtrade.CALL_PER_PERIOD=12  # Period default is 60 seconds


contrade.py initialized


# Trade profile

Choose country and years of interest

In [4]:
country_of_interest = comtrade.m49_angola
country_of_interest_desc = comtrade.COUNTRY_CODES[country_of_interest]
years_of_interest = comtrade.year_range(2020, 2021)   
print(country_of_interest_desc, years_of_interest)

Angola 2020,2021


## Global comercial balance

In [5]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format


global_trade = comtrade.get_global_stats(country_of_interest, years=years_of_interest,
                                         exports_from_word_imports=True)


In [6]:
cols = ['refYear','flowDesc','countryDesc','reporterDesc','primaryValue']
global_trade[cols]

,refYear,flowDesc,countryDesc,reporterDesc,primaryValue
0,2020,Imports,Angola,Angola,"9,337,925,708.64"
0,2020,Exports,Angola,World,"24,009,441,917.40"
1,2021,Imports,Angola,Angola,"11,378,927,920.70"
1,2021,Exports,Angola,World,"33,477,533,120.02"


In [7]:
import pandas as pd

trade_balance = pd.pivot_table(global_trade, index=['refYear','countryDesc',],columns='flowDesc', values='primaryValue').fillna(0)
trade_balance['trade_balance'] = trade_balance['Exports'] - trade_balance['Imports']
trade_balance.reset_index()

flowDesc,refYear,countryDesc,Exports,Imports,trade_balance
0,2020,Angola,"24,009,441,917.40","9,337,925,708.64","14,671,516,208.76"
1,2021,Angola,"33,477,533,120.02","11,378,927,920.70","22,098,605,199.32"


## Exportações: Destinos

Usamos as importações declaradas pelos parceiros para obter as
exportações do país de interesse.

In [8]:
show_only_top=5
years_of_interest = comtrade.year_range(2011, 2021)

# we do not specificy reporterCode, so we get all countries to report imports from country_of_interest
top_partners = comtrade.top_partners(reporterCode=None,
                                     partnerCode=country_of_interest,
                                     years=years_of_interest,
                                     flowCode='M', 
                                     rank_filter=show_only_top)

/workspaces/cipf-comtrade/comtrade.py:370: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")


In [9]:
import pandas as pd

pd.options.display.float_format='{:,.2f}'.format

cols=['reporterDesc','refYear','flowDesc','partnerDesc','rank_reporter','primaryValue', 'perc_year_flow']
top_partners[top_partners.rank_reporter<=show_only_top].sort_values(by=['refYear','flowDesc','primaryValue'], ascending=[True,True,False])[cols]

,reporterDesc,refYear,flowDesc,partnerDesc,rank_reporter,primaryValue,perc_year_flow
23,China,2011,Import,Angola,1,"24,922,180,492.00",0.38
170,United States of America,2011,Import,Angola,2,"13,833,012,040.00",0.21
117,India,2011,Import,Angola,3,"6,005,484,849.00",0.09
87,"Asia, not elsewhere specified",2011,Import,Angola,4,"5,665,315,729.00",0.09
20,Canada,2011,Import,Angola,5,"2,469,608,910.00",0.04
195,China,2012,Import,Angola,1,"33,561,896,917.00",0.44
330,United States of America,2012,Import,Angola,2,"10,030,264,550.00",0.13
278,India,2012,Import,Angola,3,"8,020,442,181.00",0.11
248,"Asia, not elsewhere specified",2012,Import,Angola,4,"5,004,958,562.00",0.07
282,South Africa,2012,Import,Angola,5,"2,802,783,009.10",0.04


In [10]:
top_export_destinations = top_partners[top_partners.rank_reporter<=show_only_top]['reporterDesc'].unique()
top_export_destinations

array(['China', 'United States of America', 'India',
       'Asia, not elsewhere specified', 'Canada', 'South Africa',
       'Portugal', 'Spain', 'France and Monaco', 'United Arab Emirates',
       'Thailand'], dtype=object)

Passamos os países para colunas, mantendo a percentagem e o valor em separado.

In [11]:
pivoted = top_partners.pivot_table(index=['refYear','flowDesc'], columns='reporterDesc', values=['primaryValue','perc_year_flow'])
pivoted.swaplevel(axis=1).fillna('',inplace=True)
pivoted.sort_index(axis=1, level=0, inplace=True)
pivoted.fillna(0)

perc_year_flow                                               \
reporterDesc            Albania Algeria Angola Antigua and Barbuda Argentina   
refYear flowDesc                                                               
2011    Import             0.00    0.00   0.00                0.00      0.00   
2012    Import             0.00    0.00   0.00                0.00      0.00   
2013    Import             0.00    0.00   0.00                0.00      0.00   
2014    Import             0.00    0.00   0.00                0.00      0.00   
2015    Import             0.00    0.00   0.00                0.00      0.00   
2016    Import             0.00    0.00   0.00                0.00      0.00   
2017    Import             0.00    0.00   0.00                0.00      0.00   
2018    Import             0.00    0.00   0.00                0.00      0.00   
2019    Import             0.00    0.00   0.00                0.00      0.00   
2020    Import             0.00    0.00   0.00                0.00      0.00   
2021    Import             0.00    0.00   0.00                0.00      0.00   

                                                                          \
reporterDesc     Armenia Asia, not elsewhere specified Australia Austria   
refYear flowDesc                                                           
2011    Import      0.00                          0.09      0.00    0.00   
2012    Import      0.00                          0.07      0.00    0.00   
2013    Import      0.00                          0.05      0.00    0.00   
2014    Import      0.00                          0.05      0.00    0.00   
2015    Import      0.00                          0.04      0.00    0.00   
2016    Import      0.00                          0.05      0.00    0.00   
2017    Import      0.00                          0.04      0.00    0.00   
2018    Import      0.00                          0.01      0.00    0.00   
2019    Import      0.00                          0.01      0.00    0.00   
2020    Import      0.00                          0.02      0.00    0.00   
2021    Import      0.00                          0.01      0.00    0.00   

                             ... primaryValue                       \
reporterDesc     Azerbaijan  ...      Ukraine United Arab Emirates   
refYear flowDesc             ...                                     
2011    Import         0.00  ...   146,120.00       301,582,058.00   
2012    Import         0.00  ...   764,720.00     1,234,968,772.00   
2013    Import         0.00  ...    63,422.00       757,063,814.73   
2014    Import         0.00  ...    11,069.00       812,542,196.16   
2015    Import         0.00  ...    50,320.00     1,056,613,911.80   
2016    Import         0.00  ...   299,907.11       956,305,699.01   
2017    Import         0.00  ...    72,378.45     1,200,168,189.47   
2018    Import         0.00  ...    20,096.32     1,569,493,453.56   
2019    Import         0.00  ...   251,112.36     1,294,307,774.80   
2020    Import         0.00  ...    35,298.67     1,018,255,600.93   
2021    Import         0.00  ...     1,389.42     1,685,701,656.52   

                                                                           \
reporterDesc       United Kingdom United States of America        Uruguay   
refYear flowDesc                                                            
2011    Import     437,170,828.00        13,833,012,040.00          16.00   
2012    Import   1,261,394,647.00        10,030,264,550.00           0.00   
2013    Import     994,286,997.00         8,922,894,861.00 315,549,401.00   
2014    Import     650,319,421.00         5,837,752,018.00 217,485,863.00   
2015    Import   1,006,251,202.00         2,917,054,295.00       1,019.00   
2016    Import     369,342,914.00         2,976,766,381.00 226,117,355.00   
2017    Import      71,778,081.35         2,686,493,698.00           0.00   
2018    Import      83,063,568.65         2,768,924,982.00 363,283,051.00   
201

In [12]:
show_only_columns = [(l1, country) for l1,country in pivoted.columns.tolist() if country in top_export_destinations]
pivoted[show_only_columns]

perc_year_flow                                 \
reporterDesc     Asia, not elsewhere specified Canada China France and Monaco   
refYear flowDesc                                                                
2011    Import                            0.09   0.04  0.38              0.03   
2012    Import                            0.07   0.03  0.44              0.02   
2013    Import                            0.05   0.02  0.44              0.02   
2014    Import                            0.05   0.02  0.48              0.03   
2015    Import                            0.04   0.01  0.43              0.04   
2016    Import                            0.05   0.00  0.49              0.04   
2017    Import                            0.04   0.00  0.56              0.01   
2018    Import                            0.01   0.00  0.58              0.01   
2019    Import                            0.01   0.00  0.63              0.01   
2020    Import                            0.02   0.00  0.61              0.02   
2021    Import                            0.01   0.00  0.62              0.01   

                                                             \
reporterDesc     India Portugal South Africa Spain Thailand   
refYear flowDesc                                              
2011    Import    0.09     0.02         0.02  0.01     0.00   
2012    Import    0.11     0.03         0.04  0.02     0.00   
2013    Import    0.09     0.05         0.03  0.04     0.00   
2014    Import    0.09     0.03         0.03  0.06     0.00   
2015    Import    0.09     0.03         0.04  0.06     0.00   
2016    Import    0.07     0.03         0.05  0.03     0.01   
2017    Import    0.11     0.01         0.04  0.03     0.01   
2018    Import    0.10     0.03         0.03  0.03     0.02   
2019    Import    0.10     0.03         0.02  0.03     0.02   
2020    Import    0.08     0.02         0.00  0.03     0.05   
2021    Import    0.07     0.00         0.01  0.02     0.05   

                                       ...     primaryValue                    \
reporterDesc     United Arab Emirates  ...           Canada             China   
refYear flowDesc                       ...                                      
2011    Import                   0.00  ... 2,469,608,910.00 24,922,180,492.00   
2012    Import                   0.02  ... 1,930,696,671.00 33,561,896,917.00   
2013    Import                   0.01  ... 1,507,292,514.00 31,972,669,347.00   
2014    Import                   0.01  ... 1,066,114,772.00 31,106,015,046.00   
2015    Import                   0.03  ...   520,271,665.00 16,001,611,383.00   
2016    Import                   0.03  ...     2,769,158.00 13,966,117,080.00   
2017    Import                   0.03  ...   130,579,986.12 20,698,709,467.00   
2018    Import                   0.04  ...       537,486.66 25,826,510,355.00   
2019    Import                   0.03  ...    21,182,570.32 23,837,402,142.00   
2020    Import                   0.04  ...        38,576.52 14,757,730,338.00   
2021    Import                   0.05  ...       316,555.00 20,908,231,317.00   

                                                                      \
reporterDesc     France and Monaco            India         Portugal   
refYear flowDesc                                                       
2011    Import    1,825,704,019.00 6,005,484,849.00 1,639,710,523.00   
2012    Import    1,199,927,582.00 8,020,442,181.00 2,290,610,271.00   
2013    Import    1,225,880,863.00 6,798,300,153.00 3,495,088,409.00   
2014    Import    2,079,376,220.00 5,641,667,702.00 2,132,511,992.00   
2015    Import    1,610,479,858.00 3,169,598,580.00 1,266,536,955.09   
2016    Import    1,029,963,745.00 2,001,936,255.00   895,881,332.31   
2017    Import      370,588,475.79 4,118,279,131.90   315,119,621.26   
2018    Import      600,213,206.89 4,337,925,217.50 1,173,908,104.09   
2019    Import      509,673,910.45 3,799,563,165.19 1,204,116,161.11   
2020    Import 

## Export as Excel

In [13]:
# export as Excel file
pivoted.to_excel(f'{country_of_interest_desc}_exports_destinations.xlsx')

In [15]:
pivoted[show_only_columns].to_excel(f'{country_of_interest_desc}_export_top_{show_only_top}_countries.xlsx')


Verificar cálculo das percentagens

In [ ]:
top_partners['perc_year_flow'].sum() / len(top_partners['refYear'].unique())

1.0

# Exportações: produtos

## Principais produtos exportados

In [16]:
show_only_top=5
years_of_interest = comtrade.year_range(2011, 2021)  
export_products = comtrade.top_commodities(reporterCode=None,
                                               partnerCode=country_of_interest,
                                               years=years_of_interest, 
                                               cmdCode='AG2',
                                               flowCode='M',  # exports from partners imports
                                           motCode=0,
                                           rank_filter=show_only_top)
print("Aggregate per product type, ordered by year, product, descreasing value")
export_products.groupby(['refYear','flowDesc','cmdCode','cmdDesc'])['primaryValue'].sum().sort_values(ascending=False).head(30)

Aggregate per product type, ordered by year, product, descreasing value


/workspaces/cipf-comtrade/comtrade.py:370: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")


refYear  flowDesc  cmdCode  cmdDesc                                                                                                                                                    
2012     Import    27       Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes                                                          71,122,194,738.19
2013     Import    27       Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes                                                          68,806,744,671.76
2011     Import    27       Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes                                                          64,243,246,424.07
2014     Import    27       Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes                                                          62,749,952,891.12


In [21]:
pd.set_option('display.max_columns',504)

cols=['refYear','partnerDesc','rank_cmd_partner','perc_partner_for_cmd','primaryValue']
cumulative_suppliers = None
for index,row in export_products[export_products['reporterDesc'].isin(top_export_destinations)].sort_values(['reporterDesc','refYear','cmdCode']).iterrows():
    print(row['refYear'],row['reporterDesc'],row['reporterCode'], f"{row['primaryValue']}",row['cmdCode'],row['cmdDesc'][:32])
    # try to get the importance of the country of interest for the partner in a given product
    suppliers = comtrade.top_partners(reporterCode=row['reporterCode'],
                                      years=row['refYear'],
                                      cmdCode=row['cmdCode'],
                                      flowCode='M',
                                      rank_filter=100)
    if cumulative_suppliers is None:
        cumulative_suppliers=suppliers
    else:
        cumulative_suppliers=pd.concat([cumulative_suppliers,suppliers])
    print(suppliers[suppliers['partnerCode']==country_of_interest][cols].sort_values('primaryValue', ascending=False))

2011 Asia, not elsewhere specified 490 1012777.0 25 Salt; sulphur; earths, stone; pl


   refYear partnerDesc  rank_cmd_partner  perc_partner_for_cmd  primaryValue
1     2011      Angola                36                  0.00  1,012,777.00
2011 Asia, not elsewhere specified 490 5663791367.0 27 Mineral fuels, mineral oils and 
   refYear partnerDesc  rank_cmd_partner  perc_partner_for_cmd  \
1     2011      Angola                 3                  0.09   

      primaryValue  
1 5,663,791,367.00  
2011 Asia, not elsewhere specified 490 212988.0 72 Iron and steel
   refYear partnerDesc  rank_cmd_partner  perc_partner_for_cmd  primaryValue
2     2011      Angola                93                  0.00    212,988.00
2011 Asia, not elsewhere specified 490 265759.0 74 Copper and articles thereof
   refYear partnerDesc  rank_cmd_partner  perc_partner_for_cmd  primaryValue
3     2011      Angola                83                  0.00    265,759.00
2011 Asia, not elsewhere specified 490 611.0 99 Commodities not specified accord
Empty DataFrame
Columns: [refYear, partnerDesc, r

In [ ]:
cumulative_suppliers.info()

NameError: name 'cumulative_suppliers' is not defined

In [ ]:
suppliers =  comtrade.export_products = comtrade.top_commodities(reporterCode=None,
                                               partnerCode=country_of_interest,
                                               years=years_of_interest, 
                                               cmdCode='AG2',
                                               flowCode='M',  # exports from partners imports
                                           motCode=0,
                                           rank_filter=show_only_top)

KeyboardInterrupt: 

In [ ]:
import pandas as pd

pd.options.display.float_format='{:,.2f}'.format
pd.options.display.max_rows = 1000

cols = ['refYear','flowDesc','cmdDesc','cmdCode','reporterDesc', 'rank_cmd','primaryValue',
        comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD,'perc_year_flow']
export_products[export_products['reporterDesc'].isin(top_export_destinations)].sort_values(['refYear','primaryValue'],ascending=[True, False])[cols]

,refYear,flowDesc,cmdDesc,cmdCode,reporterDesc,rank_cmd,primaryValue,perc_cmd_for_partner,perc_partner_for_cmd,perc_year_flow
195,2020,Import,"Mineral fuels, mineral oils and products of th...",27,China,1,"14,686,950,196.00",0.61,0.66,0.61
2319,2020,Import,"Mineral fuels, mineral oils and products of th...",27,India,1,"2,005,413,125.03",0.08,0.09,0.08
2640,2020,Import,"Mineral fuels, mineral oils and products of th...",27,Thailand,1,"1,250,575,334.49",0.05,0.06,0.05
2680,2020,Import,"Natural, cultured pearls; precious, semi-preci...",71,United Arab Emirates,1,"1,015,602,701.83",0.04,0.72,0.04
2503,2020,Import,"Mineral fuels, mineral oils and products of th...",27,Spain,1,"604,026,342.70",0.03,0.03,0.03
1275,2020,Import,"Mineral fuels, mineral oils and products of th...",27,"Asia, not elsewhere specified",1,"496,824,439.89",0.02,0.02,0.02
710,2020,Import,"Mineral fuels, mineral oils and products of th...",27,France and Monaco,1,"495,407,183.67",0.02,0.02,0.02
2775,2020,Import,"Mineral fuels, mineral oils and products of th...",27,United States of America,1,"482,834,261.00",0.02,0.02,0.02
1282,2020,Import,"Mineral fuels, mineral oils and products of th...",27,Portugal,1,"424,369,024.42",0.02,0.02,0.02
2321,2020,Import,"Mineral fuels, mineral oils and products of th...",27,South Africa,1,"35,607,163.15",0.00,0.00,0.00


In [ ]:
export_products[comtrade.PERC_CMD_IN_PARTNER].sum()

1.9928746283221535

## Principais produtos exportados e destinos

In [56]:
show_only_top = 5
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='X',
                                           motCode=0,
                                           rank_filter=show_only_top)

TypeError: top_commodities() got an unexpected keyword argument 'extra_cols'

In [21]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc',
        comtrade.PERC_PARTNER_IN_CMD, 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER ]

show_only_greater_than = export_products[comtrade.PERC_PARTNER_IN_CMD]>0.01
export_products[show_only_greater_than][cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,perc_partner_for_cmd,primaryValueFormated,perc_cmd_for_partner
2216,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",1,China,63.93%,"20,022,774,116.55",99.82%
4743,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2,India,9.28%,"2,907,767,269.85",96.66%
5779,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",4,Thailand,3.21%,"1,004,404,810.25",99.96%
2222,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",5,Singapore,2.40%,"751,652,390.95",93.65%
4827,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",6,Spain,2.01%,"628,140,238.31",79.11%
1684,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",7,Italy,2.00%,"625,729,106.51",99.00%
5824,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",9,Chile,1.65%,"516,869,632.99",99.73%
2213,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",8,South Africa,1.56%,"489,628,802.00",94.28%
2343,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",10,Netherlands,1.46%,"456,193,575.29",95.57%
4506,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",12,Indonesia,1.42%,"443,927,852.71",99.93%


## Principais **parceiros** das exportações e produtos exportados

In [14]:
show_only_top = 5
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='X',
                                           motCode=0, 
                                           rank_filter=5)

In [15]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner',	'partnerDesc','rank_cmd', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]
export_partners[cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, 
                                                                                       comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,rank_cmd,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
2216,Angola,2021,Export,1,China,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","20,022,774,116.55",99.82%,63.93%
5170,Angola,2021,Export,1,China,6,"Salt; sulphur; earths, stone; plastering materials, lime and cement","11,961,705.37",0.06%,25.51%
4696,Angola,2021,Export,1,China,5,Wood and articles of wood; wood charcoal,"11,645,905.65",0.06%,23.44%
890,Angola,2021,Export,1,China,13,"Ores, slag and ash","6,616,586.45",0.03%,66.65%
1100,Angola,2021,Export,1,China,18,Aluminium and articles thereof,"966,966.75",0.00%,17.60%
4743,Angola,2021,Export,2,India,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","2,907,767,269.85",96.66%,9.28%
3057,Angola,2021,Export,2,India,2,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin","100,217,986.40",3.33%,6.52%
1864,Angola,2021,Export,2,India,3,"Ships, boats and floating structures","106,755.20",0.00%,0.01%
1254,Angola,2021,Export,2,India,6,"Salt; sulphur; earths, stone; plastering materials, lime and cement","70,361.90",0.00%,0.15%
586,Angola,2021,Export,2,India,73,Pulp of wood or other fibrous cellulosic material; recovered (waste and scrap) paper or paperboard,"46,527.16",0.00%,99.32%


# Importações

## Principais produtos importados

In [22]:
show_only_top=10
export_products = comtrade.top_commodities(country_of_interest,
                                               partnerCode=0,  # 0 = all partners
                                               years=years_of_interest, 
                                               flowCode='M',
                                           motCode=0,
                                           rank_filter=10)


In [23]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdCode','cmdDesc', 'primaryValueFormated',
        'partnerDesc',
        comtrade.PERC_CMD_IN_PARTNER ]
export_products[cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdCode,cmdDesc,primaryValueFormated,partnerDesc,perc_cmd_for_partner
402,Angola,2021,Import,1,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","1,886,757,547.56",World,16.58%
230,Angola,2021,Import,2,84,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","1,809,939,647.27",World,15.91%
126,Angola,2021,Import,3,87,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","779,123,345.04",World,6.85%
367,Angola,2021,Import,4,85,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","776,747,585.90",World,6.83%
161,Angola,2021,Import,5,10,Cereals,"584,957,170.07",World,5.14%
27,Angola,2021,Import,6,73,Iron or steel articles,"536,792,046.75",World,4.72%
362,Angola,2021,Import,7,39,Plastics and articles thereof,"405,040,625.98",World,3.56%
240,Angola,2021,Import,8,02,Meat and edible meat offal,"348,250,700.18",World,3.06%
322,Angola,2021,Import,9,30,Pharmaceutical products,"322,068,826.04",World,2.83%
169,Angola,2021,Import,10,15,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","311,937,457.18",World,2.74%


## Principais produtos importados e origens

In [24]:
show_only_top = 10
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='M',
                                           cmdCode='AG2',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [25]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_products[comtrade.PERC_PARTNER_IN_CMD] > 0.05  # show only partners with more than 5% of trade
export_products[filter_relevant_partners][cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
3954,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",4,Togo,"687,679,076.53",100.00%,36.45%
13156,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",8,Netherlands,"318,111,982.79",73.62%,16.86%
13215,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",3,India,"291,213,622.52",39.05%,15.43%
3776,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",17,Malta,"184,307,230.08",99.00%,9.77%
4069,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",11,Belgium,"113,736,727.71",29.24%,6.03%
7421,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",1,China,"294,737,063.36",17.45%,16.28%
7026,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",2,Portugal,"219,815,557.96",16.19%,12.14%
1351,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",9,Italy,"206,406,850.29",48.33%,11.40%
7219,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",6,United Kingdom,"165,889,250.91",30.75%,9.17%
1111,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",7,United States of America,"159,642,149.65",30.52%,8.82%


### Principal origem das importações

In [32]:
show_ony_top=20
top_partners = comtrade.top_partners(country_of_interest,years_of_interest,
                                     flowCode='M', 
                                    rank_filter=show_ony_top)

In [33]:
cols=['reporterDesc','refYear','flowDesc','rank_partner','partnerDesc','primaryValueFormated', comtrade.PERC_PARTNER_IN_CMD]
top_partners[cols].sort_values(by=['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,primaryValueFormated,perc_partner_for_cmd
258,Angola,2021,Import,1,China,"1,688,904,641.34",14.84%
95,Angola,2021,Import,2,Portugal,"1,357,979,633.71",11.93%
323,Angola,2021,Import,3,India,"745,723,527.22",6.55%
319,Angola,2021,Import,4,Togo,"687,689,058.08",6.04%
495,Angola,2021,Import,5,Brazil,"546,044,463.65",4.80%
610,Angola,2021,Import,6,United Kingdom,"539,546,476.65",4.74%
251,Angola,2021,Import,7,United States of America,"523,006,089.83",4.60%
455,Angola,2021,Import,8,Netherlands,"432,100,821.51",3.80%
192,Angola,2021,Import,9,Italy,"427,046,959.44",3.75%
36,Angola,2021,Import,10,France and Monaco,"411,360,213.52",3.62%


## Principais **parceiros** das importações e produtos importados

In [34]:
show_only_top = 10
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='M',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [35]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner','partnerDesc', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_partners[comtrade.PERC_PARTNER_IN_CMD] > 0.0  # show only partners with more than 5% of trade
export_partners[filter_relevant_partners][cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
7421,Angola,2021,Import,1,China,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","294,737,063.36",17.45%,16.28%
1837,Angola,2021,Import,1,China,"Textiles, made up articles; sets; worn clothing and worn textile articles; rags","207,543,201.07",12.29%,80.04%
5764,Angola,2021,Import,1,China,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","188,027,402.48",11.13%,24.21%
3683,Angola,2021,Import,1,China,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","170,241,136.72",10.08%,21.85%
12591,Angola,2021,Import,1,China,Iron and steel,"94,179,850.47",5.58%,64.83%
5371,Angola,2021,Import,1,China,Plastics and articles thereof,"90,509,512.27",5.36%,22.35%
11724,Angola,2021,Import,1,China,Glass and glassware,"62,248,517.42",3.69%,75.37%
6889,Angola,2021,Import,1,China,Iron or steel articles,"60,344,496.23",3.57%,11.24%
6008,Angola,2021,Import,1,China,Rubber and articles thereof,"58,498,228.59",3.46%,43.76%
11490,Angola,2021,Import,1,China,Pharmaceutical products,"48,373,928.61",2.86%,15.02%


## Análise de simetrias nos dados

In [36]:


partnerA = country_of_interest


### Principais destinos de exportação reportados pelo país

In [83]:
show_only_top = 30
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='X',
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=None,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     customsCode=None,
                     echo_url=False
                     )
cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000][cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format}) 


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
348,Angola,China,World,2021,Export,"20,058,426,271.91",20058426271.912998,58.19%,All modes of transport,True,All customs procedure codes
211,Angola,India,World,2021,Export,"3,008,390,595.07",3008390595.072000,8.73%,All modes of transport,True,All customs procedure codes
288,Angola,United Arab Emirates,World,2021,Export,"1,399,449,578.20",1399449578.198000,4.06%,All modes of transport,True,All customs procedure codes
165,Angola,Thailand,World,2021,Export,"1,004,833,529.77",1004833529.770000,2.91%,All modes of transport,True,All customs procedure codes
46,Angola,Singapore,World,2021,Export,"802,659,408.76",802659408.761000,2.33%,All modes of transport,True,All customs procedure codes
109,Angola,Spain,World,2021,Export,"793,986,998.99",793986998.994000,2.30%,All modes of transport,True,All customs procedure codes
67,Angola,Italy,World,2021,Export,"632,025,040.54",632025040.539000,1.83%,All modes of transport,True,All customs procedure codes
279,Angola,South Africa,World,2021,Export,"519,358,721.33",519358721.333000,1.51%,All modes of transport,True,All customs procedure codes
70,Angola,Chile,World,2021,Export,"518,264,559.23",518264559.233000,1.50%,All modes of transport,True,All customs procedure codes
63,Angola,Netherlands,World,2021,Export,"477,332,019.79",477332019.785000,1.38%,All modes of transport,True,All customs procedure codes


### Principais destinos de exportação segundo o registo da importações do resto do mundo

In [82]:
show_only_top=30
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='M',
                     cmdCode='TOTAL',
                     reporterCode=None,
                     partnerCode=partnerA,
                     partner2Code=0,
                     motCode = 0,
                     # customsCode=0,
                     period=years_of_interest,
                     echo_url=False
                     )
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>1000][cols].sort_values(by=['refYear','primaryValue','reporterDesc',], ascending=[True,False,True]).head(show_only_top).style.format({'perc': '{:,.2%}'.format}) 

,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
29,China,Angola,World,2021,Import,"20,908,231,317.00",20908231317.000000,62.45%,All modes of transport,True,All customs procedure codes
324,India,Angola,World,2021,Import,"2,315,947,568.41",2315947568.408000,6.92%,All modes of transport,True,All customs procedure codes
388,United Arab Emirates,Angola,World,2021,Import,"1,685,701,656.52",1685701656.518000,5.04%,All modes of transport,True,All customs procedure codes
385,Thailand,Angola,World,2021,Import,"1,607,756,231.51",1607756231.510000,4.80%,All modes of transport,True,All customs procedure codes
413,United States of America,Angola,World,2021,Import,"1,089,250,402.00",1089250402.000000,3.25%,All modes of transport,True,All customs procedure codes
325,Singapore,Angola,World,2021,Import,"752,550,761.40",752550761.398000,2.25%,All modes of transport,True,All customs procedure codes
355,Spain,Angola,World,2021,Import,"739,484,782.77",739484782.768000,2.21%,All modes of transport,True,All customs procedure codes
5,Belgium,Angola,World,2021,Import,"615,024,126.87",615024126.871000,1.84%,All modes of transport,True,All customs procedure codes
28,Chile,Angola,World,2021,Import,"537,823,552.96",537823552.960000,1.61%,All modes of transport,True,All customs procedure codes
147,Indonesia,Angola,World,2021,Import,"463,466,072.00",463466072.000000,1.38%,All modes of transport,True,All customs procedure codes


## Análise de simetrias nos dados com um país específico

In [104]:
partnerB = comtrade.COUNTRY_CODES_REVERSE['China']
flowCode = 'M'
flowDesc = comtrade.FLOWS_CODES[flowCode]


phrase = f"{comtrade.COUNTRY_CODES.get(partnerA)} {flowDesc} to {comtrade.COUNTRY_CODES.get(partnerB)} reported by {comtrade.COUNTRY_CODES.get(partnerA)}" \
 if flowCode == 'X' else f"{comtrade.COUNTRY_CODES.get(partnerA)} {flowDesc} from {comtrade.COUNTRY_CODES.get(partnerB)} reported by {comtrade.COUNTRY_CODES.get(partnerA)}"

cmd_detail = 'AG4'
show_only_top = 10

cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','cmdDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 

In [105]:
df_import_rep_a = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode=cmd_detail,
                     reporterCode=partnerA,
                     partnerCode=partnerB,
                     partner2Code=0,
                     period=2020,
                     motCode = 0,
                     echo_url=False
                     )


In [106]:
if df_import_rep_a is not None:
    total_import = df_import_rep_a['primaryValue'].sum()
    df_import_rep_a['perc'] = df_import_rep_a['primaryValue']/total_import
    print(phrase)
    result = df_import_rep_a[cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print("No",phrase)
    result = ''

result

Angola Import from China reported by Angola


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,cmdDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
1021,Angola,China,World,2020,Import,"Rail locomotives; (other than those of heading no. 8601), locomotive tenders","106,385,404.28",106385404.277000,7.42%,All modes of transport,True,All customs procedure codes
1382,Angola,China,World,2020,Import,"New pneumatic tyres, of rubber","50,669,463.76",50669463.757000,3.53%,All modes of transport,True,All customs procedure codes
1087,Angola,China,World,2020,Import,nan,"48,169,359.41",48169359.409000,3.36%,All modes of transport,True,All customs procedure codes
1722,Angola,China,World,2020,Import,"Iron or non-alloy steel; flat-rolled products, width 600mm or more, clad, plated or coated","40,716,106.16",40716106.155000,2.84%,All modes of transport,True,All customs procedure codes
508,Angola,China,World,2020,Import,"Electric transformers, static converters (e.g. rectifiers) and inductors","39,653,621.92",39653621.919000,2.76%,All modes of transport,True,All customs procedure codes
2340,Angola,China,World,2020,Import,"Iron or non-alloy steel; flat-rolled products of a width of 600mm or more, hot-rolled, not clad, plated or coated","39,392,570.45",39392570.454000,2.75%,All modes of transport,True,All customs procedure codes
381,Angola,China,World,2020,Import,"Telephone sets, including smartphones and other telephones for cellular/wireless networks; other apparatus for the transmission or reception of voice, images or other data (including wired/wireless networks), excluding items of 8443, 8525, 8527, or 8528","36,628,189.17",36628189.175000,2.55%,All modes of transport,True,All customs procedure codes
28,Angola,China,World,2020,Import,"Medicaments; (not goods of heading no. 3002, 3005 or 3006) consisting of mixed or unmixed products for therapeutic or prophylactic use, put up in measured doses (incl. those in the form of transdermal admin. systems) or packed for retail sale","29,087,289.30",29087289.296000,2.03%,All modes of transport,True,All customs procedure codes
2147,Angola,China,World,2020,Import,"Air conditioning machines; comprising a motor driven fan and elements for changing the temperature and humidity, including those machines in which the humidity cannot be separately regulated","22,047,690.54",22047690.539000,1.54%,All modes of transport,True,All customs procedure codes
2671,Angola,China,World,2020,Import,"Clock, watch and similar glasses, glasses for non-corrective or corrective spectacles, curved, bent, hallowed etc, not optically worked; hollow glass spheres and their segments for manufacture","21,833,158.76",21833158.762000,1.52%,All modes of transport,True,All customs procedure codes


In [108]:
# invertemos

print(years_of_interest, partnerA, partnerB)

flowCodeInv = 'X' if flowCode == 'M' else 'M'
flowDesc = comtrade.FLOWS_CODES[flowCodeInv]


phrase = f"{comtrade.COUNTRY_CODES.get(partnerB)} {flowDesc} to {comtrade.COUNTRY_CODES.get(partnerA)} reported by {comtrade.COUNTRY_CODES.get(partnerB)}" if flowCodeInv == 'X'\
    else f"{comtrade.COUNTRY_CODES.get(partnerB)} {flowDesc} from {comtrade.COUNTRY_CODES.get(partnerA)} reported by {comtrade.COUNTRY_CODES.get(partnerB)}"


print(phrase)

df_import_rep_b = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCodeInv,
                     cmdCode=cmd_detail,
                     reporterCode=partnerB,
                     partnerCode=partnerA,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
if df_import_rep_b is not None:
    total_import = df_import_rep_b['primaryValue'].sum()
    df_import_rep_b['perc'] = df_import_rep_b['primaryValue']/total_import
    result = df_import_rep_b[cols].sort_values(by=['refYear','reporterDesc','primaryValue'], ascending=[True,True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print("No",phrase)
    result = ''
result

2021 24 156
China Export to Angola reported by China


ReadTimeout: HTTPSConnectionPool(host='comtradeapi.un.org', port=443): Read timed out. (read timeout=10)